In [277]:
import numpy as np
import pandas as pd
import bs4 as bs
import requests
import re

In [278]:
base = 'https://dangkeebs.com'

In [279]:
url = 'https://dangkeebs.com/collections/keycaps?page='

In [280]:
i = 1
paths = []
while True:
    data = requests.get(url + str(i))
    print(url + str(i))
    html = bs.BeautifulSoup(data.text, 'html.parser')
    if len(html.find_all(attrs={"class": "grid__item grid__item--collection-template small--one-half medium-up--one-quarter"})) == 0:
        break
    for link in html.find_all('a'):
        ref = link.get('href')
        if "products" in ref:
            paths.append(ref)
    i += 1

https://dangkeebs.com/collections/keycaps?page=1
https://dangkeebs.com/collections/keycaps?page=2


In [281]:
temp = pd.DataFrame(index=range(len(paths)))
temp.loc[0, 4] = 3

In [282]:
result = pd.DataFrame(index=range(len(paths)))

for i, path in enumerate(paths):
    link = base + path
    curr = requests.get(link)
    text = bs.BeautifulSoup(curr.text, 'html.parser')
    
    for tag in text.find_all('meta'):
        temp = set()
        
        property = tag.get('property')
        if property != None and property not in temp:
            result.loc[i, property[3:]] = tag.get('content')
            temp.add(property)
            
    for span in text.find_all('span'):
        if "data-add-to-cart-text" in span.attrs:
            result.loc[i, "availability"] = str(span.contents).strip("[\n]\\n' ")
            break
    for option in text.find_all('option'):
        if "selected" in option.attrs:
            result.loc[i, "kit"] = str(option.contents).strip("[\n]\\n' ")

In [283]:
result

,site_name,url,title,type,description,price:amount,price:currency,image,image:secure_url,availability,kit
0,Dangkeebs,https://dangkeebs.com/products/epbt-flower,[Pre-Order] EPBT Flower,product,This keycap set was inspired by a holiday take...,15.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Sold out,Base - Sold out
1,Dangkeebs,https://dangkeebs.com/products/group-buy-gmk-c...,[Group-Buy] GMK Cluck,product,"Introducing GMK Cluck, inspired by chickens, o...",22.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Sold out,Base - Sold out
2,Dangkeebs,https://dangkeebs.com/products/group-buy-epbt-...,[Group-Buy] EPBT Hellas,product,A keycap set inspired by the beautiful ancient...,17.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Sold out,Base - Sold out
3,Dangkeebs,https://dangkeebs.com/products/group-buy-gmk-d...,[Group-Buy] GMK Daifuku,product,Disclaimer: The final base kit will include a ...,20.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Sold out,Hiragana Base Kit - Sold out
4,Dangkeebs,https://dangkeebs.com/products/group-buy-gmk-h...,[Group-Buy] GMK Hanok,product,30% of the profits will be donated to Hate Is ...,22.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Sold out,Base - Sold out
5,Dangkeebs,https://dangkeebs.com/products/thok-diamond-ar...,THOK Diamond Artisan,product,DSA profile White resin infill Colour-shift po...,45.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Add to cart,Default Title
6,Dangkeebs,https://dangkeebs.com/products/pbtfans-white-o...,PBTFans White on Black (WoB),product,PBTFans White on Black is a classic set that's...,18.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Add to cart,Base
7,Dangkeebs,https://dangkeebs.com/products/pbtfans-dolch,PBTFans Dolch,product,PBTFans Dolch reimagines a timeless keycap col...,18.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Add to cart,Base
8,Dangkeebs,https://dangkeebs.com/products/pbtfans-kabuki-cho,PBTFans Kabuki-Cho,product,PBTFans Kabuki-Cho is inspired by the entertai...,18.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Add to cart,40s
9,Dangkeebs,https://dangkeebs.com/products/epbt-cloudy-day,EPBT Cloudy Day,product,Specifications: ABS plastic Double-shot legend...,69.00,USD,http://cdn.shopify.com/s/files/1/0064/2560/828...,https://cdn.shopify.com/s/files/1/0064/2560/82...,Sold out,Base Kit - Sold out


In [159]:
result = result[result["availability"] != "Sold out"]
result = result[result["kit"].str.contains("Base") | result["kit"].str.contains("Default")]

In [160]:
result = result[["site_name", "url", "title", "description", "price:amount", "image"]]

In [161]:
result["in_stock"] = np.where(result["title"].str.contains("[", regex=False), 0, 1)

In [163]:
result.to_json("test.json")

In [164]:
result['json'] = result.apply(lambda x: x.to_json(), axis=1)

In [249]:
def get_links(url):
    url = url + "?page="
    i = 1
    paths = []
    while True:
        data = requests.get(url + str(i))
        print(url + str(i))
        html = bs.BeautifulSoup(data.text, 'html.parser')
        if len(html.find_all(attrs={"class": "grid__item grid__item--collection-template small--one-half medium-up--one-quarter"})) == 0:
            break
        for link in html.find_all('a'):
            ref = link.get('href')
            if "products" in ref:
                paths.append(ref)
        i += 1
        
    return paths
    

def get_keycaps(base, url):
    paths = get_links(url)

    result = pd.DataFrame(index=range(len(paths)))
    for i, path in enumerate(paths):
        link = base + path
        curr = requests.get(link)
        text = bs.BeautifulSoup(curr.text, 'html.parser')
        
        for tag in text.find_all('meta'):
            temp = set()
            
            property = tag.get('property')
            if property != None and property not in temp:
                result.loc[i, property[3:]] = tag.get('content')
                temp.add(property)
                
        for span in text.find_all('span'):
            if "data-add-to-cart-text" in span.attrs:
                result.loc[i, "availability"] = str(span.contents).strip("[\n]\\n' ")
                break
        for option in text.find_all('option'):
            if "selected" in option.attrs:
                result.loc[i, "kit"] = str(option.contents).strip("[\n]\\n' ")

    print(result.columns)
    
    return result




In [250]:
temp = get_keycaps('https://dangkeebs.com', 'https://dangkeebs.com/collections/switches')

https://dangkeebs.com/collections/switches?page=1
https://dangkeebs.com/collections/switches?page=2
https://dangkeebs.com/collections/switches?page=3
https://dangkeebs.com/collections/switches?page=4
https://dangkeebs.com/collections/switches?page=5
https://dangkeebs.com/collections/switches?page=6
Index(['site_name', 'url', 'title', 'type', 'description', 'price:amount',
       'price:currency', 'image', 'image:secure_url', 'availability', 'kit'],
      dtype='object')


In [256]:
temp.to_csv("tmep.csv")

In [258]:
x = requests.get("https://dangkeebs.com/collections/switches/products/s1-peaches?variant=43114917069021")

In [260]:
b = bs.BeautifulSoup(x.text)

In [273]:
def get_links(url):
    url = url + "?page="

    paths = []
    for i in range(1, 11):
        data = requests.get(url + str(i))
        html = bs.BeautifulSoup(data.text, 'html.parser')
        for link in html.find_all('a'):
            ref = link.get('href')
            print(ref)
            if ref != None and "products" in ref:
                paths.append(ref)
        
    return paths

In [274]:
get_links("https://spaceholdings.net/collections/keycaps")

#
#
#
javascript:void(0)
javascript:void(0)
/
#
#
https://www.facebook.com/spacecables
https://twitter.com/space_cables
https://instagram.com/spacecables
/account/register
/account/login
/
javascript:void(0)
#
/collections/keyboards
https://spaceholdings.net/collections/keycaps
/collections/keycaps-in-stock
/collections/keycap-group-buy
/collections/lubricant
/collections/deskpads
/collections/cables
https://spaceholdings.net/collections/lemo-cables
https://spaceholdings.net/collections/aviator-cables
#
https://support.spaceholdings.net/
https://discord.gg/spacecables
/collections/past-sales
/

/collections/keycaps/products/gmk-lavender
/collections/keycaps/products/gmk-lavender
/collections/keycaps/products/gmk-lavender
/collections/keycaps/products/gmk-nightshade
/collections/keycaps/products/gmk-nightshade
/collections/keycaps/products/gmk-nightshade
/collections/keycaps/products/gmk-pacific
/collections/keycaps/products/gmk-pacific
/collections/keycaps/products/gmk-pacific
/collect

['/collections/keycaps/products/gmk-lavender',
 '/collections/keycaps/products/gmk-lavender',
 '/collections/keycaps/products/gmk-lavender',
 '/collections/keycaps/products/gmk-nightshade',
 '/collections/keycaps/products/gmk-nightshade',
 '/collections/keycaps/products/gmk-nightshade',
 '/collections/keycaps/products/gmk-pacific',
 '/collections/keycaps/products/gmk-pacific',
 '/collections/keycaps/products/gmk-pacific',
 '/collections/keycaps/products/hibi-jwst-novelty-keycap',
 '/collections/keycaps/products/hibi-jwst-novelty-keycap',
 '/collections/keycaps/products/hibi-jwst-novelty-keycap',
 '/collections/keycaps/products/gmk-purple-night',
 '/collections/keycaps/products/gmk-purple-night',
 '/collections/keycaps/products/gmk-purple-night',
 '/collections/keycaps/products/gmk-hooty',
 '/collections/keycaps/products/gmk-hooty',
 '/collections/keycaps/products/gmk-hooty',
 '/collections/keycaps/products/gmk-relegendables-plus',
 '/collections/keycaps/products/gmk-relegendables-plus'